In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_excel("Wine_filtered.xlsx")
data.head()

,Pais,Continente,Provincia,region_1,Descripcion,Sentimiento,Viñedo_especifico,Variedad,Bodega,Año,Antiguedad,Puntos,Precio,Ratio_puntos_precio
0,Italy,Europa,Sicily & Sardinia,Etna,aroma include tropical fruit broom brimstone d...,Positiva,Vulkà Bianco,White Blend,Nicosia,2013,11,87,28,3.107143
1,Portugal,Europa,Douro,Unknown,ripe fruity wine smooth structured firm tannin...,Positiva,Avidagos,Portuguese Red,Quinta dos Avidagos,2011,13,87,15,5.800000
2,US,America del norte,Michigan,Lake Michigan Shore,pineapple rind lemon pith orange blossom start...,Positiva,Reserve Late Harvest,Riesling,St. Julian,2013,11,87,13,6.692308
3,US,America del norte,Oregon,Willamette Valley,like regular bottling come rough tannic rustic...,Positiva,Vintner's Reserve Wild Child Block,Pinot Noir,Sweet Cheeks,2012,12,87,65,1.338462
4,Spain,Europa,Northern Spain,Navarra,blackberry raspberry aroma typical navarran wh...,Positiva,Ars In Vitro,Tempranillo-Merlot,Tandem,2011,13,87,15,5.800000


In [10]:
df = data.filter(["Pais", "Provincia", "Sentimiento", "Variedad", "Bodega", "Año", "Antiguedad", "Puntos", "Precio", "Ratio_puntos_precio"], axis = 1)
df.head(2)

,Pais,Provincia,Sentimiento,Variedad,Bodega,Año,Antiguedad,Puntos,Precio,Ratio_puntos_precio
0,Italy,Sicily & Sardinia,Positiva,White Blend,Nicosia,2013,11,87,28,3.107143
1,Portugal,Douro,Positiva,Portuguese Red,Quinta dos Avidagos,2011,13,87,15,5.800000


In [11]:
df = df.groupby('Pais').apply(lambda x: x.assign(count=x['Pais'].count())).sort_values(by='count', ascending=False).reset_index(drop=True).drop(columns='count', axis=1) 
df.head(2)

C:\Users\Positivo BGH\AppData\Local\Temp\ipykernel_2624\1990950117.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Pais').apply(lambda x: x.assign(count=x['Pais'].count())).sort_values(by='count', ascending=False).reset_index(drop=True).drop(columns='count', axis=1)


,Pais,Provincia,Sentimiento,Variedad,Bodega,Año,Antiguedad,Puntos,Precio,Ratio_puntos_precio
0,US,Michigan,Positiva,Riesling,St. Julian,2013,11,87,13,6.692308
1,US,California,Negativa,Zinfandel,Hendry,2012,12,90,35,2.571429


In [12]:
X = df.drop("Precio", axis = 1)
y = df.Precio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [27]:
categorical_columns = ['Pais', 'Provincia', 'Sentimiento', 'Variedad', 'Bodega']
numerical_columns = ['Antiguedad', 'Puntos', 'Ratio_puntos_precio']

# Definir transformaciones para cada grupo de columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), categorical_columns),
        ('num', MinMaxScaler(), numerical_columns)
    ]
)

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Aplicar el pipeline a los datos de entrenamiento
X_train_transformed = pipeline.fit_transform(X_train)

# Convertir el resultado en un DataFrame para visualizarlo fácilmente
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=categorical_columns + numerical_columns)

X_train_transformed_df.head(2)

,Pais,Provincia,Sentimiento,Variedad,Bodega,Antiguedad,Puntos,Ratio_puntos_precio
0,38.0,48.0,0.0,465.0,4679.0,0.076190,0.25,0.182222
1,38.0,350.0,2.0,108.0,9549.0,0.038095,0.35,0.163914


In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [28]:
model = RandomForestRegressor().fit(X_train_transformed_df, y_train)

In [29]:
X_test_transformed = pipeline.fit_transform(X_test)

# Convertir el resultado en un DataFrame para visualizarlo fácilmente
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=categorical_columns + numerical_columns)

In [30]:
y_pred = model.predict(X_test_transformed_df)

In [33]:
r2_score(y_test, y_pred)

0.9343590893882951

In [34]:
import joblib

In [35]:
joblib.dump(model, 'model.pkl')

# Cargar el modelo desde el archivo
#model_cargado = joblib.load('model.pkl')

['model.pkl']